<a href="https://colab.research.google.com/github/ShedovaNastya/Face-Recognition-Project/blob/main/IdentificatonRateMetric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Создадим два набора изображений лиц: query и distractors. Никакие лица из этих наборов не должны содержаться в обучающем и валидационном датасете.
Посчитаем косинусные расстояния между лицами, соответствующими одним и тем же людям из query части. Например, пусть одному человеку соответствуют три фото в query: 01.jpg, 02.jpg, 03.jpg. Тогда считаем три косинусных расстояния между всеми тремя парами из этих фото.
посчитаем косинусные расстояния между лицами, соответствующими разным людям из query части.
посчитаем косинусные расстояния между всеми парами лиц из query и distractors. Т.е. пара — это (лицо из query, лицо из distractors). Всего получится |query|*|distractors| пар.
Сложим количества пар, полученных на 2 и 3 шагах. Это количество false пар.
Зафиксируем FPR (false positive rate). Пусть, например, будет 0.01. FPR, умноженный на количество false пар из шага 4 — это разрешенное количество false positives, которые мы разрешаем нашей модели. Обозначим это количество через N.
Отсортируем все значения косинусных расстояний false пар. N — ое по счету значение расстояния зафиксируем как пороговое расстояние.
Посчитаем количество positive пар с шага 1, которые имеют косинусное расстояние меньше, чем пороговое расстояние. Поделим это количество на общее количество positive пар с шага 1. Это будет TPR (true positive rate) — итоговое значение нашей метрики.
Такая метрика обычно обозначается как TPR@FPR=0.01. FPR может быть разным. Приразных FPR будет получаться разное TPR.

Смысл этой метрики в том, что мы фиксируем вероятность ошибки вида false positive, т.е. когда "сеть сказала, что это один и тот же человек, но это не так", считаем порог косинусного расстояния для этого значения ошибки, потом берем все positive пары и смотрим, у скольких из них расстояние меньше этого порога. Т.е. насколько точно наша сеть ищет похожие лица при заданной вероятности ошибки вида false positive.

#IMPORT MODULE

In [5]:
!pip install torchmetrics

import torch
import os
from IPython.display import clear_output
import numpy as np
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.models import list_models, get_model
from torchmetrics.functional import pairwise_cosine_similarity
from tqdm.notebook import tqdm as bar
from copy import deepcopy
from collections import defaultdict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [41]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# !unzip /content/drive/MyDrive/celebA_train_500.zip
!unzip /content/drive/MyDrive/celebA_ir.zip
clear_output()

#Identificaton rate metric

In [43]:
f = open('./celebA_ir/celebA_anno_query.csv', 'r')
query_lines = f.readlines()[1:]
f.close()
query_lines = [x.strip().split(',') for x in query_lines]
# plain list of image names from query. Neede to compute embeddings for query
query_img_names = ['./celebA_ir/celebA_query/' + x[0] for x in query_lines]

query_dict = defaultdict(list)
for img_name, img_class in query_lines:# словарь ключ- id, значения- изображения, принадлежащие id
  query_dict[img_class].append('./celebA_ir/celebA_query/' + img_name)

distractors_img_names =['./celebA_ir/celebA_distractors/' + i for i in os.listdir('./celebA_ir/celebA_distractors')]#хранятся названия изображений из distractors


In [ ]:
query_dict

In [45]:
class id_rate_dataset(torch.utils.data.Dataset):
  def __init__(self, images_list, transform):
    self.images_list = images_list
    self.transform = transform
  def __len__(self):
    return len(self.images_list)

  def __getitem__(self, idx):
    image_name = self.images_list[idx]
    image = Image.open(image_name)
    image = self.transform(image)
    return image




In [46]:
# Модель для оценки
m1 = get_model("efficientnet_b0", weights="IMAGENET1K_V1")
m1.classifier[1] = torch.nn.Linear(in_features=1280, out_features=500, bias=True)
device = 'cpu'
m1 = m1.to(device)


filename = '/content/drive/MyDrive/eff_net_param_m1.pth'
m1.load_state_dict(torch.load(filename, map_location=device))

m1.classifier = m1.classifier[:1]


In [47]:
!du -h celebA_ir/celebA_query/

4.9M	celebA_ir/celebA_query/


In [48]:
test_query_dict = {
    2876: ['1.jpg', '2.jpg', '3.jpg'],
    5674: ['5.jpg'],
    864:  ['9.jpg', '10.jpg'],
}
test_query_img_names = ['1.jpg', '2.jpg', '3.jpg', '5.jpg', '9.jpg', '10.jpg']
test_query_embeddings = torch.tensor([
                    [1.56, 6.45,  -7.68],
                    [-1.1 , 6.11,  -3.0],
                    [-0.06,-0.98,-1.29],
                    [8.56, 1.45,  1.11],
                    [0.7,  1.1,   -7.56],
                    [0.05, 0.9,   -2.56],
])

test_distractors_img_names = ['11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg']

test_distractors_embeddings = torch.tensor([
                    [0.12, -3.23, -5.55],
                    [-1,   -0.01, 1.22],
                    [0.06, -0.23, 1.34],
                    [-6.6, 1.45,  -1.45],
                    [0.89,  1.98, 1.45],
])


In [49]:
def compute_embeddings(model, images_list):#функция считает выход ембеддингового слоя
  '''
  compute embeddings from the trained model for list of images.
  params:
    model: trained nn model that takes images and outputs embeddings
    images_list: list of images paths to compute embeddings for
  output:
    list: list of model embeddings. Each embedding corresponds to images
          names from images_list
  '''
  out_data = []
  dataset = id_rate_dataset(images_list,  T.Compose([T.Resize((224, 224)), T.ToTensor()]))
  loader = DataLoader(dataset, batch_size = 30, shuffle = False)

  model.eval()
  correct = 0
  with torch.no_grad():
    for data in loader:
      data = data.to(device)

      out_data.append(model(data))
  return torch.cat(out_data)


#compute_embeddings(m1, query_img_names)

317653
[ 0.9419636   0.81423724  0.80625534 ... -0.11651252 -0.11936088
 -0.12310144]
317653
[ 0.9419636   0.81423724  0.80625534 ... -0.11651252 -0.11936088
 -0.12310144]
317653
[ 0.9419636   0.81423724  0.80625534 ... -0.11651252 -0.11936088
 -0.12310144]
317653
[ 0.9419636   0.81423724  0.80625534 ... -0.11651252 -0.11936088
 -0.12310144]


In [51]:
def cosine_similarity(list_em, list_em2):
  em1 = torch.Tensor(list_em)
  em2 = torch.Tensor(list_em2)
  similarity= pairwise_cosine_similarity(em1,em2)
  return similarity

In [53]:
a = torch.randn(size=(3,3))
cosine_similarity(a,a)

tensor([[ 1.0000, -0.2525, -0.7671],
        [-0.2525,  1.0000, -0.3054],
        [-0.7671, -0.3054,  1.0000]])

In [54]:
def compute_cosine_query_pos(query_dict, query_img_names, query_embeddings):
  '''
  compute cosine similarities between positive pairs from query (stage 1)
  params:
    query_dict: dict {class: [image_name_1, image_name_2, ...]}. Key: class in
                the dataset. Value: images corresponding to that class
    query_img_names: list of images names
    query_embeddings: list of embeddings corresponding to query_img_names
  output:
    list of floats: similarities between embeddings czorresponding
                    to the same people from query list

    '''
  similarity = []
  images = {name:idx for idx, name in enumerate(query_img_names)}#словарь ключ - имя изображения, значение - индекс в query_img_names
  for id in query_dict.keys():#идем по каждому лицу
    emb = []
    img_id = query_dict[id]#получаю список изображений, принадлежащих одному лицу
    for img in img_id:# иду по кааждому изображению
      emb.append(query_embeddings[images[img]].unsqueeze(0))#добавляю в список ембеддинг каждой фотки, unsqeeze - добавляет размерность фиктивную [[...]] [[...]]
    emb = torch.cat(emb)#[[...],[...]]
    matrix = cosine_similarity(emb, emb)
    mask = torch.triu(torch.ones_like(matrix), diagonal=1)# triu- верх.треугольник, ones_like - все единицы, diagonal=1 - по нулям
    matrix = matrix[mask == 1]
    if len(matrix) > 0:# обрабатываем случай когда у одного лица лолько одно изображение,добавлять тогда не будем
      similarity.append(matrix)
  return torch.cat(similarity)


# cosine_query_pos = compute_cosine_query_pos(query_dict, query_img_names,
#                                             query_embeddings)
test_cosine_query_pos = compute_cosine_query_pos(test_query_dict, test_query_img_names,
                                            test_query_embeddings)
true_cosine_query_pos = torch.Tensor([0.8678237233650096, 0.21226104378511604,
                         -0.18355866977496182, 0.9787437979250561])
assert np.allclose(sorted(test_cosine_query_pos), sorted(true_cosine_query_pos)), \
      "A mistake in compute_cosine_query_pos function"
print(test_cosine_query_pos)

tensor([ 0.8678,  0.2123, -0.1836,  0.9787])


In [55]:
def compute_cosine_query_neg(query_dict, query_img_names, query_embeddings):
  '''
  compute cosine similarities between negative pairs from query (stage 2)
  params:
    query_dict: dict {class: [image_name_1, image_name_2, ...]}. Key: class in
                the dataset. Value: images corresponding to that class
    query_img_names: list of images names
    query_embeddings: list of embeddings corresponding to query_img_names
  output:
    list of floats: similarities between embeddings corresponding
                    to different people from query list
  '''
  similarity = []
  keys_id = list(query_dict.keys())
  images = {name:idx for idx, name in enumerate(query_img_names)}#словарь ключ - имя изображения, значение - индекс в query_img_names
  for id1 in range(len(keys_id)):
    emb1 = []
    images1 = query_dict[keys_id[id1]]
    for img in images1:
      emb1.append(query_embeddings[images[img]].unsqueeze(0))
    emb1 = torch.cat(emb1)
    for id2 in range(id1+1,len(keys_id)):
      images2 = query_dict[keys_id[id2]]
      emb2 = []
      for img in images2:
        emb2.append(query_embeddings[images[img]].unsqueeze(0))
      emb2 = torch.cat(emb2)
      # print(cosine_similarity(emb1, emb2))
      similarity.append(cosine_similarity(emb1, emb2).view(-1))#здесь разворачиваем двумерный тензор в массивчик тензоров через view
      # break
    # break

  return torch.cat(similarity)



test_cosine_query_neg = compute_cosine_query_neg(test_query_dict, test_query_img_names,
                                            test_query_embeddings)



true_cosine_query_neg = [0.15963231223161822, 0.8507997093616965, 0.9272761484302097,
                         -0.0643994061127092, 0.5412660901220571, 0.701307100338029,
                         -0.2372575528216902, 0.6941032794522218, 0.549425446066643,
                         -0.011982733001947084, -0.0466679194884999]
print(test_cosine_query_neg)
assert np.allclose(sorted(test_cosine_query_neg), sorted(true_cosine_query_neg)), \
      "A mistake in compute_cosine_query_neg function"


tensor([ 0.1596, -0.0644, -0.2373,  0.8508,  0.9273,  0.5413,  0.7013,  0.6941,
         0.5494, -0.0120, -0.0467])


In [56]:
def compute_cosine_query_distractors(query_embeddings, distractors_embeddings):
  '''
  compute cosine similarities between negative pairs from query and distractors
  (stage 3)
  params:
    query_embeddings: list of embeddings corresponding to query_img_names
    distractors_embeddings: list of embeddings corresponding to distractors_img_names
  output:
    list of floats: similarities between pairs of people (q, d), where q is
                    embedding corresponding to photo from query, d —
                    embedding corresponding to photo from distractors
  '''
  similarity = []
  batch_size = 50
  for q in range(0, len(query_embeddings), batch_size):
    emb1 = query_embeddings[q: q + batch_size]
    for d in range(0,len(distractors_embeddings), batch_size):
      emb2 = distractors_embeddings[d:d+batch_size]
      similarity.append(cosine_similarity(emb1, emb2).view(-1))
  return torch.cat(similarity)



  raise NotImplementedError


test_cosine_query_distractors = compute_cosine_query_distractors(test_query_embeddings,
                                                            test_distractors_embeddings)


true_cosine_query_distractors = [0.3371426578637511, -0.6866465610863652, -0.8456563512871669,
                                 0.14530087113136106, 0.11410510307646118, -0.07265097629002357,
                                 -0.24097699660707042,-0.5851992679925766, 0.4295494455718534,
                                 0.37604478596058194, 0.9909483738948858, -0.5881093317868022,
                                 -0.6829712976642919, 0.07546364489032083, -0.9130970963915521,
                                 -0.17463101988684684, -0.5229363015558941, 0.1399896725311533,
                                 -0.9258034013399499, 0.5295114163723346, 0.7811585442749943,
                                 -0.8208760031249596, -0.9905139680301821, 0.14969764653247228,
                                 -0.40749654525418444, 0.648660814944824, -0.7432584300096284,
                                 -0.9839696492435877, 0.2498741082804709, -0.2661183373780491]
assert np.allclose(sorted(test_cosine_query_distractors), sorted(true_cosine_query_distractors)), \
      "A mistake in compute_cosine_query_distractors function"

In [57]:
test_cosine_query_pos

tensor([ 0.8678,  0.2123, -0.1836,  0.9787])

In [63]:
def compute_ir(cosine_query_pos, cosine_query_neg, cosine_query_distractors,
               fpr=0.1):
  '''
  compute identification rate using precomputer cosine similarities between pairs
  at given fpr
  params:
    cosine_query_pos: cosine similarities between positive pairs from query
    cosine_query_neg: cosine similarities between negative pairs from query
    cosine_query_distractors: cosine similarities between negative pairs
                              from query and distractors
    fpr: false positive rate at which to compute TPR
  output:
    float: threshold for given fpr
    float: TPR at given FPR
  '''
  cnt_false = len(cosine_query_neg) + len(cosine_query_distractors)
  n = int(cnt_false * fpr)
  false_pars = np.concatenate([cosine_query_neg.cpu().numpy(), cosine_query_distractors.cpu().numpy()])
  false_pars = np.sort(false_pars)[::-1]
  threshold = false_pars[n]
  # print(n)
  # print(false_pars)
  cosine_query_pos = cosine_query_pos.cpu().numpy()
  num_true_positive = np.sum(cosine_query_pos >= threshold)
  return threshold, num_true_positive/ len(cosine_query_pos)

In [59]:
test_thr = []
test_tpr = []
for fpr in [0.5, 0.3, 0.1]:
  x, y = compute_ir(test_cosine_query_pos, test_cosine_query_neg,
                    test_cosine_query_distractors, fpr=fpr)
  test_thr.append(x)
  test_tpr.append(y)

20
[ 0.9909484   0.9272762   0.8507998   0.78115857  0.70130706  0.69410324
  0.6486608   0.5494254   0.5412661   0.52951145  0.4295494   0.37604478
  0.33714268  0.24987411  0.15963233  0.14969766  0.1453009   0.13998966
  0.11410506  0.07546364 -0.01198272 -0.04666792 -0.06439941 -0.072651
 -0.17463103 -0.23725754 -0.24097699 -0.2661184  -0.4074966  -0.5229363
 -0.5851992  -0.5881093  -0.68297124 -0.68664664 -0.7432585  -0.82087606
 -0.84565634 -0.91309714 -0.9258034  -0.98396957 -0.9905139 ]
12
[ 0.9909484   0.9272762   0.8507998   0.78115857  0.70130706  0.69410324
  0.6486608   0.5494254   0.5412661   0.52951145  0.4295494   0.37604478
  0.33714268  0.24987411  0.15963233  0.14969766  0.1453009   0.13998966
  0.11410506  0.07546364 -0.01198272 -0.04666792 -0.06439941 -0.072651
 -0.17463103 -0.23725754 -0.24097699 -0.2661184  -0.4074966  -0.5229363
 -0.5851992  -0.5881093  -0.68297124 -0.68664664 -0.7432585  -0.82087606
 -0.84565634 -0.91309714 -0.9258034  -0.98396957 -0.9905139 ]


In [60]:
true_thr = [-0.011982733001947084, 0.3371426578637511, 0.701307100338029]
assert np.allclose(np.array(test_thr), np.array(true_thr)), "A mistake in computing threshold"

true_tpr = [0.75, 0.5, 0.5]
assert np.allclose(np.array(test_tpr), np.array(true_tpr)), "A mistake in computing tpr"

In [61]:
test_thr

[-0.011982724, 0.33714268, 0.70130706]

In [66]:
# query_embeddings = compute_embeddings(m1, query_img_names)
# distractors_embeddings = compute_embeddings(m1, distractors_img_names)

cosine_query_pos = compute_cosine_query_pos(query_dict, query_img_names, query_embeddings)
cosine_query_neg = compute_cosine_query_neg(query_dict, query_img_names, query_embeddings)
cosine_query_distractors = compute_cosine_query_distractors(query_embeddings, distractors_embeddings)

thr = []
tpr = []

for fpr in [0.5, 0.1, 0.05, 0.01]:#0.01 при 1% когда он допускает разных людей одинаковыми, то при 0,44 верно предсказывает на изображении одного и того же человека
  x, y = compute_ir(cosine_query_pos, cosine_query_neg, cosine_query_distractors, fpr)
  thr.append(x)
  tpr.append(y)

In [67]:
tpr

[0.9703824468446437,
 0.7847972284491542,
 0.6785544460294817,
 0.4408667889409687]